# C3M1: Peer Reviewed Assignment

### Outline:
The objectives for this assignment:

1. Apply Binomial regression methods to real data.
2. Understand how to analyze and interpret binomial regression models.
3. Flex our math skills by determining whether certain distributions are members of the exponential family.

General tips:

1. Read the questions carefully to understand what is being asked.
2. This work will be reviewed by another human, so make sure that you are clear and concise in what your explanations and answers.

In [1]:
# Load required libraries
library(tidyverse)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Problem 1: Binomial (Logistic) Regression

The National Institute of Diabetes and Digestive and Kidney Diseases conducted a study of 768 adult female Pima Indians living near Phoenix, AZ. The purpose of the study was to investigate the factors related to diabetes. 

*Before we analyze these data, we should note that some have raised ethical issues with its collection and popularity in the statistics and data science community. We should think seriously about these concerns. For example, Maya Iskandarani wrote a brief [piece](https://researchblog.duke.edu/2016/10/24/diabetes-and-privacy-meet-big-data/) on consent and privacy concerns raised by this dataset. After you familarize yourself with the data, we'll then turn to these ethical concerns.*


First, we'll use these data to get some practice with GLM and Logistic regression.

In [2]:
# Load the data
pima = read.csv("pima.txt", sep="\t")
# Here's a description of the data: https://rdrr.io/cran/faraway/man/pima.html
head(pima)

,pregnant,glucose,diastolic,triceps,insulin,bmi,diabetes,age,test
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1
6,5,116,74,0,0,25.6,0.201,30,0


### 1. (a) Data Cleaning? What about Data Scrubbing? Data Sterilizing?

This is a real data set, which means that there's likely going to be gaps and missing values in the data. Before doing any modeling, we should inspect the data and clean it if necesary.

Perform simple graphical and numerical summaries of the data. Pay attention for missing or nonsensical values. Can you find any obvious irregularities? If so, take appropriate steps to correct these problems. In the markdown cell, specify what cleaning you did and why you did it.

Finally, split your data into training and test sets. Let the training set contain $80\%$ of the rows and the test set contain the remaining $20\%$.

In [3]:
# Step 3: Check for missing values
missing_summary <- sapply(pima, function(x) sum(is.na(x)))
print(missing_summary)

# Step 4: Impute missing values (use mean imputation as an example)
pima_clean <- pima
for (col in names(pima_clean)) {
  if (is.numeric(pima_clean[[col]])) {
    pima_clean[[col]][is.na(pima_clean[[col]])] <- mean(pima_clean[[col]], na.rm = TRUE)
  }
}

# Step 5: Summary of cleaned data
summary(pima_clean)

# Step 6: Split the data into train (80%) and test (20%)
set.seed(2024) # Set seed for reproducibility
train_indices <- sample(1:nrow(pima_clean), size = 0.8 * nrow(pima_clean))
train_data <- pima_clean[train_indices, ]
test_data <- pima_clean[-train_indices, ]

# Verify the split
cat("Training data size:", nrow(train_data), "\n")
cat("Testing data size:", nrow(test_data), "\n")

 pregnant   glucose diastolic   triceps   insulin       bmi  diabetes       age 
        0         0         0         0         0         0         0         0 
     test 
        0 


    pregnant         glucose        diastolic         triceps     
 Min.   : 0.000   Min.   :  0.0   Min.   :  0.00   Min.   : 0.00  
 1st Qu.: 1.000   1st Qu.: 99.0   1st Qu.: 62.00   1st Qu.: 0.00  
 Median : 3.000   Median :117.0   Median : 72.00   Median :23.00  
 Mean   : 3.845   Mean   :120.9   Mean   : 69.11   Mean   :20.54  
 3rd Qu.: 6.000   3rd Qu.:140.2   3rd Qu.: 80.00   3rd Qu.:32.00  
 Max.   :17.000   Max.   :199.0   Max.   :122.00   Max.   :99.00  
    insulin           bmi           diabetes           age       
 Min.   :  0.0   Min.   : 0.00   Min.   :0.0780   Min.   :21.00  
 1st Qu.:  0.0   1st Qu.:27.30   1st Qu.:0.2437   1st Qu.:24.00  
 Median : 30.5   Median :32.00   Median :0.3725   Median :29.00  
 Mean   : 79.8   Mean   :31.99   Mean   :0.4719   Mean   :33.24  
 3rd Qu.:127.2   3rd Qu.:36.60   3rd Qu.:0.6262   3rd Qu.:41.00  
 Max.   :846.0   Max.   :67.10   Max.   :2.4200   Max.   :81.00  
      test      
 Min.   :0.000  
 1st Qu.:0.000  
 Median :0.000  
 

Training data size: 614 
Testing data size: 154 


### 1. (b) Initial GLM modelling


Our data is clean and we're ready to fit! What kind of model should we use to fit these data? Notice that the `test` variable is either $0$ or $1$, for whether the individual tested positive for diabetes. Because `test` is binary, we should use logistic regression (which is a kind of binomial regression).

Fit a model with `test` as the response and all the other variables as predictors. Can you tell whether this model fits the data?

In [4]:
# logit model
logit_model <- glm(test ~ ., data = train_data, family = binomial)

# summarize the model fit
summary(logit_model)



Call:
glm(formula = test ~ ., family = binomial, data = train_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4798  -0.7300  -0.4281   0.7420   2.9493  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -8.2130001  0.7785903 -10.549  < 2e-16 ***
pregnant     0.1183929  0.0357879   3.308 0.000939 ***
glucose      0.0352638  0.0041847   8.427  < 2e-16 ***
diastolic   -0.0130875  0.0057262  -2.286 0.022281 *  
triceps     -0.0009585  0.0075277  -0.127 0.898678    
insulin     -0.0009091  0.0009840  -0.924 0.355526    
bmi          0.0860673  0.0166329   5.175 2.29e-07 ***
diabetes     0.7810790  0.3211506   2.432 0.015010 *  
age          0.0152288  0.0102926   1.480 0.138982    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 796.42  on 613  degrees of freedom
Residual deviance: 583.55  on 605  degrees of freedom
AIC: 601.55

Numbe

         Actual
Predicted  0  1
        0 92 22
        1 10 30
Accuracy: 0.7922078 


### 1. (c) Remember Bayes

A quick analytical interlude.

Is diastolic blood pressure significant in the regression model? Do women who test positive have higher diastolic blood pressures? Explain the distinction between the two questions and discuss why the answers are only apparently contradictory.

In [ ]:
# The first question is about significance and the p-value appear to be lower than 0.05. 
# The second question is about the effect which would be higher for women than men. 
# There might be some correlated variable producing this effect.

### 1. (d) GLM Interpretation

We've seen so many regression summaries up to this point, how is this one different from all the others? Well, to really understand any model, it can be helpful to loop back and plug the fitted results back into the model's mathematical form.

Explicity write out the equation for the binomial regression model that you fit in (b). Then, in words, explain how a $1$ unit change of `glucose` affects `test`, assuming all other predictors are held constant.

In [5]:
# test =−8.213+0.118⋅pregnant+0.035⋅glucose−0.013⋅diastolic−0.001⋅triceps−0.001⋅insulin+0.086⋅bmi+0.781⋅diabetes+0.015⋅age
# The coefficient for glucose is 0.0350.035, meaning:
# For a 1-unit increase in glucose, holding all other predictors constant, the log-odds of testing positive (test=1test=1) increase by 0.0350.035.
# In terms of probabilities, a 1-unit increase in glucose increases the odds of testing positive by a factor of exp⁡(0.035)≈1.036exp(0.035)≈1.036, or about 3.6%.

### 1. (e) GLM Prediction

One of the downsides of Logistic Regression is that there isn't an easy way of evaulating the goodness of fit of the model without predicting on new data. But, if we have more data to test with, then there are many methods of evaluation to use. One of the best tools are confusion matrices, which (despite the name) are actually not that hard to understand.

A confusion matrix compares the predicted outcomes of a Logistic Regression Model (or any classification model) with the actual classifications. For binary classification, it is a $2 \times 2$ matrix where the rows are the models' predicted outcome and the columns are the actual classifications. An example is displayed below.

|  | True | False |  
| --- | --- | --- |
| 1 | 103 | 37 |  
| 0 | 55  | 64 |  

In the example, we know the following information:
* The [1,1] cell is the number of datapoints that were correctly predicted to be $1$. The value (103) is the number of True Positives (TP). 
* The [2,2] cell is the number of datapoints that were correctly predicted to be $0$. The value is the number of True Negatives (TN).
* The [1, 2] cell is the number of datapoints that were predicted to be $1$ but where actually $0$. This is the number of False Positives (FP), also called Type I error. In the context of our diabetes dataset, this would mean our model predicted that the person would have diabetes, but they actually did not.
* The [2, 1] cell is the number of datapoints that were predicted to be $0$ but where actually $1$. This is the number of False Negatives (FN), also called Type 2 error. In the context of our diabetes dataset, this would mean our model predicted that the person would not have diabetes, but they actually did have diabetes.

Use your model to predict the outcomes of the test set. Then construct a confusion matrix for these predictions and display the results.

In [6]:
# make predictions on the test set
predicted_probabilities <- predict(logit_model, newdata = test_data, type = "response")
predicted_classes <- ifelse(predicted_probabilities > 0.5, 1, 0)

# assess the fit using confusion matrix
confusion_matrix <- table(Predicted = predicted_classes, Actual = test_data$test)
print(confusion_matrix)

         Actual
Predicted  0  1
        0 92 22
        1 10 30
Accuracy: 0.7922078 


### 1. (f) Evaluation Statistics

Using the four values from the confusion matrix, we can construct evaulation statistics to get a numerical approximation for our model's performance. Spend some time researching accuracy, precision, recall and F score. 

Calculate these values for your model's predictions on the test set. Clearly display your results. How well do you think your model fits the data?

In [7]:
# calculate metrics like accuracy
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat("Accuracy:", accuracy, "\n")

# Extract components of the confusion matrix
TP <- confusion_matrix["1", "1"]  # True Positives
FP <- confusion_matrix["1", "0"]  # False Positives
FN <- confusion_matrix["0", "1"]  # False Negatives
TN <- confusion_matrix["0", "0"]  # True Negatives

# Compute Precision
precision <- TP / (TP + FP)

# Compute Recall
recall <- TP / (TP + FN)

# Compute F1-Score
f1_score <- 2 * (precision * recall) / (precision + recall)

# Print results
cat("Precision:", precision, "\n")
cat("Recall:", recall, "\n")
cat("F1-Score:", f1_score, "\n")

# The model achieves good precision (0.75), indicating it makes relatively few false positive predictions, 
# but recall is lower (0.5769), reflecting some missed positive cases, resulting in a moderate F1-score of 0.6522, 
# balancing precision and recall.

Accuracy: 0.7922078 
Precision: 0.75 
Recall: 0.5769231 
F1-Score: 0.6521739 


### 1. (g) Understanding Evaluation Statistics

Answer the following questions in the markdown cell below.

1. Give an example scenario for when accuracy would be a misleading evaulation statistic.
2. Confusion matrices can also be used for non-binary classification problems. Describe what a confusion matrix would look like for a response with $3$ levels.
3. You'll have to take our word on the fact (or spend some time researching) that Type I error and Type II error are inversely related. That is, if a model is very good at detecting false positives, then it will be bad at detecting false negatives. In the case of our diabetes dataset, would you prefer a model that overestimates the Type 1 error or overestimates the Type II error. Justify your answer.

1. Example Scenario for Misleading Accuracy:

Accuracy can be misleading in scenarios with imbalanced datasets. For example, in a medical diagnosis where 95% of patients are healthy, a model that always predicts "healthy" achieves 95% accuracy but fails to identify any sick patients.
2. Confusion Matrix for 3 Levels:

For a response with 3 levels (e.g., A, B, C), a confusion matrix would be a 3x3 table where rows represent the predicted class, and columns represent the actual class. Each cell (i,j)(i,j) shows the number of instances predicted as class ii when the true class is jj.
3. Type I vs. Type II Error Preference:

In the diabetes dataset, it is preferable to overestimate Type I error (false positives) because detecting diabetes early (even if some healthy individuals are flagged) is critical for timely intervention, whereas missing true cases (Type II error) could lead to severe health consequences.

### 1. (h) Ethical Issues in Data Collection

Read Maya Iskandarani's [piece](https://researchblog.duke.edu/2016/10/24/diabetes-and-privacy-meet-big-data/) on consent and privacy concerns raised by this dataset. Summarize those concerns here.

The main concern raised is the ethical dilemma of using historical medical data, such as the Pima Indian Diabetes Data set, which was collected decades ago without consent for future uses, raising issues of privacy, informed consent, and the evolving nature of medical research.

## Problem 2: Practicing those Math skills

One of the conditions of GLMs is that the "random component" of the data needs to come from the Exponential Family of Distributions. But how do we know if a distribution is in the Exponential Family? Well, we could look it up. Or we could be proper mathematicians and check the answer ourselves! Let's flex those math muscles.

### 2. (a) But it's in the name...

Show that $Y \sim exponential(\lambda)$, where $\lambda$ is known, is a member of the exponential family.

The pdf can be written in the exponential family form as f(y;λ)=exp(log(λ)−λy)

### 2. (b) Why can't plants do math? Because it gives them square roots!

Let $Y_i \sim exponential(\lambda)$ where $i \in \{ 1, \dots, n\}$. Then $Z = \sum_{i=1}^n Y_i \sim Gamma(n, \lambda)$. Show that $Z$ is also a member of the exponential family.

The pdf can also be written in the exponential family form, here as: f(z;n,λ)=(n−1)!λnzn−1exp(−λz)